In [9]:
"""Should be written like this end result.."""
# from speechbrain.inference.classifiers import  EncoderClassifier
# 
# classifier = EncoderClassifier.from_hparams(source="./XVector_Augmented_SpeakerCounter_results/XVector/Augmented/1986/save/CKPT+2024-03-23+10-00-05+00", 
#                                             savedir="./sampleinference")
# 
# # Load an audio file
# signal, fs = torchaudio.load("./maindata/eval/session_0_spk_3/session_0_spk_3_mixture.wav")
# 
# # Classify the audio to get the number of speakers and overlap information
# prediction = classifier.classify_batch(signal)
# prediction

'Should be written like this end result..'

In [3]:
%%file SpeakerCounter.py

import torch
import torchaudio
from speechbrain.pretrained import Pretrained
from speechbrain.utils.data_utils import split_path
from speechbrain.utils.fetching import fetch

class SpeakerOverlapDetector(Pretrained):
    """
    A class for detecting speaker count and overlaps in audio segments.
    This custom class is inspired by SpeechBrain's VAD and classification interfaces.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.sample_rate = self.hparams.sample_rate

    def load_audio(self, path, start=0, end=None):
        """Load a segment of an audio file."""
        waveform, fs = torchaudio.load(path, frame_offset=start, num_frames=end)
        if fs != self.sample_rate:
            waveform = torchaudio.transforms.Resample(orig_freq=fs, new_freq=self.sample_rate)(waveform)
        return waveform

    def segment_audio(self, signal, segment_length=2.0, overlap=0.5):
        """Segment audio into overlapping chunks."""
        segment_length_samples = int(segment_length * self.sample_rate)
        overlap_samples = int(overlap * self.sample_rate)
        segments = []

        start = 0
        while start + segment_length_samples <= signal.size(1):
            end = start + segment_length_samples
            segments.append(signal[:, start:end])
            start += (segment_length_samples - overlap_samples)
        
        if len(segments) == 0:
            segments.append(signal)
        
        return torch.stack(segments, dim=0)

    def predict_speakers(self, segments):
        """Predict the number of speakers in each segment."""
        # Placeholder for actual model prediction
        # Replace this with your model's prediction logic
        predictions = torch.randint(0, 3, (segments.shape[0],))
        return predictions

    def detect_speaker_changes(self, audio_path):
        """Detect changes in speaker count in an audio file."""
        signal = self.load_audio(audio_path)
        segments = self.segment_audio(signal)
        predictions = self.predict_speakers(segments)

        changes = []
        for i in range(1, len(predictions)):
            if predictions[i] != predictions[i-1]:
                changes.append((i, predictions[i].item()))
        
        return changes

    def process_audio_file(self, audio_path):
        """Process an audio file to detect speaker overlaps and changes."""
        changes = self.detect_speaker_changes(audio_path)
        for i, num_speakers in changes:
            print(f"Change detected at segment {i}: {num_speakers} speakers")


Overwriting SpeakerCounter.py


In [ ]:
detector = SpeakerOverlapDetector.from_hparams(source="your_model_directory", savedir="your_savedir")
detector.process_audio_file("path_to_your_audio_file.wav")